In [2]:
from math import pi
import pandas as pd
from bokeh.palettes import Category20c, Category20
from bokeh.transform import cumsum
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource,ranges,LabelSet
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import curdoc
from bokeh.io import export_png
from bokeh.io import output_notebook
import numpy as np

# output_file("colormapped_bars.html")
curdoc().theme = 'dark_minimal'
categories = ["Food","Transportation","Investing","Amazon","Other","Subscription","Zelle"]
colors = ['#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b','#c49c94',
'#e377c2','#f7b6d2','#7f7f7f','#c7c7c7','#bcbd22','#dbdb8d']


def csv_to_df():
    df = pd.read_csv("/Users/benjaminadame/Desktop/test.csv")
    return df

def monthly_expenses():
    #Reading the csv file and creating a new dataframe
    df = csv_to_df()
    data = [df['Description'],df['Debit']]
    headers = ['Description','Debit']
    df2 = pd.concat(data,axis = 1,keys = headers)
    df2 = df2.dropna()
    g1 = df2.groupby(['Description']).sum()
    g1 = g1.groupby( [ "Description", "Debit"] ).count().reset_index()
    print("Monthly Expenses: ")
    print(g1)

    #Plot the chart
    x = dict(g1.values)
    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Category20c[len(x)]
    p = figure(plot_height=350, title="August 2021 Expenses", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))
    p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    show(p)

def daily_expenses():

    df = csv_to_df()
    data = [df['Date'],df['Description'], df['Debit']]
    headers = ['Date','Description', 'Debit']
    df2 = pd.concat(data, axis=1, keys=headers)

    #Group by date
    df2 = df2.groupby(['Date']).sum()
    df2 = df2.groupby(["Date", "Debit"]).count().reset_index()


    #Create List
    dates = df2['Date'].tolist()
    debits = df2["Debit"].tolist()

    #Visualize the data
    source = ColumnDataSource(data=dict(dates=dates, debits=debits))
    p = figure(x_range=dates, plot_height=400, title="August Daily Expenses",
               toolbar_location=None, tools="")


    p.vbar(x=dates, top=debits, width=0.9)


    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = pi/3
    p.xaxis.subgroup_label_orientation = "normal"
    p.xaxis.group_label_orientation = 0.8
    show(p)

def multicategorical_chart(categories):
    dates = []
    amount = [0,0,0,0,0,0,0]

    #Set up data
    df = csv_to_df()
    df2 = df[['Date', 'Description', 'Debit']].copy().dropna().reset_index().drop(["index"],axis = 1)
    cols = ['Date','Description']
    grouped_df = df2.groupby(cols)[['Debit']].sum().reset_index()
    my_dict ={}
    g = grouped_df.groupby('Date')

    #Create Dictionary
    for date, date_df in g:
        date_df = date_df[['Description','Debit']].copy()
        dates.append(date)
        desc = date_df['Description'].to_list()
        deb  = date_df['Debit'].to_list()
        for i in range(len(desc)):
            for j in range(len(categories)):
                if desc[i] == categories[j]:
                    amount[j]=deb[i]
        my_dict[date] = amount
        amount = [0,0,0,0,0,0,0]

    my_dict['Categories'] = categories

    p = figure(y_range=categories, plot_height=600,plot_width = 900, x_range=(0, 300), title="Daily Expenditure for August",
               toolbar_location=None)

    p.hbar_stack(dates, y='Categories', height=0.7, color=colors, source=ColumnDataSource(my_dict),
                 legend_label=["%s " % x for x in dates])



    p.y_range.range_padding = .5
    p.ygrid.grid_line_color = None
    p.legend.location = "top_right"
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.click_policy = "hide"
    p.legend.label_text_font_size = "8px"
    show(p)

def accrued_debits_credits():

    #Set up data
    #Credits
    color = ['#aec7e8','#ff7f0e']
    df = csv_to_df()
    df2 = [df['Date'],df['Credit']]
    headers = ['Date', 'Amount']
    df2 = pd.concat(df2, axis=1, keys=headers)
    df2 = df2.dropna()
    df2 = df2.groupby(['Date']).sum()
    df2 = df2.groupby(["Date", "Amount"]).count().reset_index()

    #Debits
    df3 = [df['Date'], df['Debit']]
    headers = ['Date', 'Amount']
    df3 = pd.concat(df3, axis=1, keys=headers)
    df3 = df3.dropna()
    df3 = df3.groupby(['Date']).sum()
    df3 = df3.groupby(["Date", "Amount"]).count().reset_index()

    #Get cumulative sum for both df2 & df3
    df2_cum_sum = df2["Amount"].to_list()
    df3_cum_sum = df3["Amount"].to_list()
    df2_cum_sum = np.cumsum(df2_cum_sum)
    df3_cum_sum = np.cumsum(df3_cum_sum)

    #Create new dataframe
    new_dataframe1 = {'Date':df2["Date"].to_list(),
                      'Amount': list(df2_cum_sum)}
    new_dataframe2 = {'Date': df3['Date'].to_list(),
                      'Amount': list(df3_cum_sum)}
    df_01 = pd.DataFrame(new_dataframe1)
    df_02 = pd.DataFrame(new_dataframe2)

    #Plot the chart
    p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
    p.title.text = 'Monthly Debit and Credit'
    for data, name, color in zip([df_01, df_02], ['Credit','Debit'], colors):
        df = pd.DataFrame(data)
        df['Date'] = pd.to_datetime(df['Date'])
        p.line(df['Date'], df['Amount'], line_width=2, color=color, alpha=0.8, legend_label=name)
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"

    show(p)

def totals_chart():
    # Set up data
    # Credits
    color = ['#aec7e8', '#ff7f0e']
    df = csv_to_df()
    df2 = [df['Date'], df['Credit']]
    headers = ['Date', 'Amount']
    df2 = pd.concat(df2, axis=1, keys=headers)
    df2 = df2.dropna()
    df2 = df2.groupby(['Date']).sum()
    df2 = df2.groupby(["Date", "Amount"]).count().reset_index()


    # Debits
    df3 = [df['Date'], df['Debit']]
    headers = ['Date', 'Amount']
    df3 = pd.concat(df3, axis=1, keys=headers)
    df3 = df3.dropna()
    df3 = df3.groupby(['Date']).sum()
    df3 = df3.groupby(["Date", "Amount"]).count().reset_index()

    #Get totals
    credit = df2['Amount'].to_list()
    debit = df3['Amount'].to_list()
    credit_sum = sum(credit)
    debit_sum = sum(debit)
    

    if (credit_sum-debit_sum)<0:
        new_df ={'Amount': [credit_sum,debit_sum,int(abs(credit_sum-debit_sum))],
              'Type':["Credit",'Debit',"Overhead"]}
        df_01 = pd.DataFrame(new_df, index=[0, 1, 2])
        df_01['angle'] = df_01['Amount'] / df_01['Amount'].sum() * 2 * pi
        df_01['color'] = Category20[3]
        source = ColumnDataSource(df_01)
        p = figure(plot_height=350, title="Total Credits, Debits and Savings", toolbar_location=None,
                   tools="hover", tooltips="@Type: @Amount{0.2f} ", x_range=(-.5, .5))

        p.annular_wedge(x=0, y=1, inner_radius=0.15, outer_radius=0.25, direction="anticlock",
                        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                        line_color="white", fill_color='color', legend_field='Type', source=source)
        p.axis.axis_label = None
        p.axis.visible = False
        p.grid.grid_line_color = None
    else:
        new_df = {'Amount': [credit_sum,debit_sum,int(abs(credit_sum-debit_sum))],
              'Type':["Credit",'Debit',"Savings"]}
        df_01 = pd.DataFrame(new_df,index=[0,1,2])
        df_01['angle'] = df_01['Amount'] / df_01['Amount'].sum() * 2 * pi
        df_01['color'] = Category20[3]
        source = ColumnDataSource(df_01)
        p = figure(plot_height=350, title="Total Credits, Debits and Savings", toolbar_location=None,
               tools="hover", tooltips="@Type: @Amount{0.2f} ", x_range=(-.5, .5))

        p.annular_wedge(x=0, y=1, inner_radius=0.15, outer_radius=0.25, direction="anticlock",
                    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                    line_color="white", fill_color='color', legend_field='Type', source=source)


        p.axis.axis_label = None
        p.axis.visible = False
        p.grid.grid_line_color = None

    show(p)

# make a grid
#grid = gridplot([[monthly_expenses(), daily_expenses()], [totals_chart(),accrued_debits_credits()]], plot_width=700, plot_height=500)


In [5]:
monthly_expenses()

Monthly Expenses: 
      Description   Debit
0             Atm   40.00
1         Clothes  163.31
2         Fitness  150.31
3            Food   47.68
4    Subscription   15.13
5  Transportation  261.75
6           Zelle   25.00


In [6]:
daily_expenses()

In [7]:
totals_chart()

In [8]:
accrued_debits_credits()